In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchnet
!pip install pytorch-ignite

In [ ]:
from __future__ import print_function, division
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
plt.ion()   # interactive mode

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

In [ ]:
ROOT = '/content/drive/MyDrive/mura_clahe'
data_dir = os.path.join(ROOT, 'mura')
images_dir = os.path.join(data_dir, 'images')
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

In [ ]:
pretrained_size = 320

pretrained_means = [0.485,0.456,0.406]

pretrained_stds= [0.229,0.224,0.225]

batch_size=8

data_transforms = {
    
    'train': transforms.Compose([
        transforms.Resize((pretrained_size,pretrained_size)),
                           transforms.RandomHorizontalFlip(),
                           transforms.RandomRotation(10),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
    ]),
    'val': transforms.Compose([
        transforms.Resize((pretrained_size,pretrained_size)),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
    ]),
                   }
print("Initializing Datasets and Dataloaders...\n")
# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True) for x in ['train', 'val']}
device = torch.device("cuda:0")
dataset_sizes ={x:len(image_datasets[x]) for x in ['train','val']}

In [ ]:
# Class names convert to index
image_datasets['train'].class_to_idx
class_names=image_datasets['train'].classes
print(">>Class Names: {}\n".format(image_datasets['train'].classes))
print(">>Class Index: {}\n".format(image_datasets['train'].class_to_idx))
print(">>Number of images in training={}\n".format(dataset_sizes['train']))
print(">>Number of images in test={}\n".format(dataset_sizes['val']))
print("    Number of steps for training set={}\n".format(len(dataloaders['train'])))
print("    Number of steps for test set={}\n".format(len(dataloaders['val']))) 
# 1:positive #0:negative

In [ ]:
train_losses = []
train_acc = []
val_losses = []
val_acc = []

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    ghost=True
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.detach() * inputs.size(0) #.item()
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss/ dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            #Losses and accuracy per epochs are stored in array for plot graphs
            if phase == 'train':              
                train_losses.append(epoch_loss)
                train_acc.append(epoch_acc)
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            if phase == 'val':
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
                val_losses.append(epoch_loss)
                val_acc.append(epoch_acc)
            torch.cuda.empty_cache()

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
        print()

        torch.cuda.empty_cache()
    # Determine total traning time
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    # Print best validation accuracy 
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(best_model_wts, "/content/drive/MyDrive/el1.pth")
    return model

In [ ]:
class SpinalNet_ResNext_50(nn.Module):
    def __init__(self):
        super(SpinalNet_ResNext_50, self).__init__()
        
        self.fc_spinal_layer1 = nn.Sequential(
            nn.Linear(1024, 20),
            nn.ReLU(inplace=True),)
        self.fc_spinal_layer2 = nn.Sequential(
            nn.Linear(1024+20, 20),
            nn.ReLU(inplace=True),)
        self.fc_spinal_layer3 = nn.Sequential(
            nn.Linear(1024+20, 20),
            nn.ReLU(inplace=True),)
        self.fc_spinal_layer4 = nn.Sequential(
            nn.Linear(1024+20, 20),
            nn.ReLU(inplace=True),)
        self.fc_out = nn.Sequential(
            nn.Linear(20*4, 2),nn.Sigmoid())
        
    def forward(self, x):

        x1 = self.fc_spinal_layer1(x[:, 0:1024])
        x2 = self.fc_spinal_layer2(torch.cat([ x[:,1024:2*1024], x1], dim=1))
        x3 = self.fc_spinal_layer3(torch.cat([ x[:,0:1024], x2], dim=1))
        x4 = self.fc_spinal_layer4(torch.cat([ x[:,1024:2*1024], x3], dim=1))
        
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.cat([x, x3], dim=1)
        x = torch.cat([x, x4], dim=1)

        
        x = self.fc_out(x)
        
        return x

In [ ]:
class SpinalNet_Dense(nn.Module):
    def __init__(self):
        super(SpinalNet_Dense, self).__init__()
        
        self.fc_spinal_layer1 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(960, 240),
            nn.BatchNorm1d(240), nn.ReLU(inplace=True),)
        self.fc_spinal_layer2 = nn.Sequential(
            nn.Dropout(p = 0.5), 
            nn.Linear(960+240, 240),
            nn.BatchNorm1d(240), 
            nn.ReLU(inplace=True),)
        self.fc_spinal_layer3 = nn.Sequential(
            nn.Dropout(p = 0.5), 
            nn.Linear(960+240, 240),
            nn.BatchNorm1d(240), 
            nn.ReLU(inplace=True),)
        self.fc_spinal_layer4 = nn.Sequential(
            nn.Dropout(p = 0.5), 
            nn.Linear(960+240, 240),
            nn.BatchNorm1d(240), 
            nn.ReLU(inplace=True),)
        self.fc_out = nn.Sequential(
            nn.Dropout(p = 0.5), 
            nn.Linear(240*4, 2),nn.Sigmoid(),)        

    def forward(self, x):
        x1 = self.fc_spinal_layer1(x[:, 0:960])
        x2 = self.fc_spinal_layer2(torch.cat([ x[:,960:2*960], x1], dim=1))
        x3 = self.fc_spinal_layer3(torch.cat([ x[:,0:960], x2], dim=1))
        x4 = self.fc_spinal_layer4(torch.cat([ x[:,960:2*960], x3], dim=1))
        
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.cat([x, x3], dim=1)
        x = torch.cat([x, x4], dim=1)

        
        x = self.fc_out(x)
        return x

In [ ]:
resnext50 =models.resnext50_32x4d(pretrained=False)
resnext50.fc =  SpinalNet_ResNext_50()
resnext50.load_state_dict(torch.load('/content/drive/MyDrive/ResNext_50_SP_FC_Clahe_Edge.pth'))
resnext50=resnext50.to(device)
resnext50.fc.fc_out=nn.Linear(80,80)

In [ ]:
densenet169=models.densenet169(pretrained=False)
num_ftrs = densenet169.classifier.in_features
densenet169.classifier = nn.Sequential(nn.Linear(num_ftrs, 2),
nn.Sigmoid())
densenet169.load_state_dict(torch.load('/content/drive/MyDrive/DenseNet_169_FC_Edge_Clahe.pth'))
densenet169=densenet169.to(device)
densenet169.classifier = nn.Linear(1664, 1664)

In [ ]:
densenet201 =models.densenet201(pretrained=False)
densenet201.classifier=SpinalNet_Dense()
densenet201.load_state_dict(torch.load('/content/drive/MyDrive/DenseNet_201_SP_FC_Edge_Clahe.pth'))
densenet201=densenet201.to(device)
densenet201.classifier.fc_out=nn.Linear(960,960)

In [ ]:
for param in resnext50.parameters():
    param.requires_grad_(False)

for param in densenet169.parameters():
    param.requires_grad_(False)
    
for param in densenet201.parameters():
    param.requires_grad_(False)

In [ ]:
class EL1(nn.Module):
    def __init__(self):
        super(EL1, self).__init__()

        self.modelA = resnext50
        self.modelB = densenet169
        self.modelC = densenet201

        self.hidden = nn.Linear(2704, 2704)
        self.classifier =nn.Linear(2704, 2)

    def forward(self, x):

        x1 = self.modelA(x.clone())
        x1 = x1.view(x1.size(0), -1)
        x2 = self.modelB(x)
        x2 = x2.view(x2.size(0), -1)
        x3 = self.modelC(x)
        x3 = x3.view(x3.size(0), -1)

        x = torch.cat((x1, x2,x3), dim=1)
        x = self.hidden(torch.softmax(x, dim=1)) 
        x = self.classifier(torch.sigmoid(x))

        return x 

In [ ]:
ensemble_1=EL1()
ensemble_1 = ensemble_1.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ensemble_1.parameters(), lr=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
el_1 = train_model(el_1, criterion, optimizer, exp_lr_scheduler,40)

In [ ]:
#This code block draw graph for accuracy
matplotlib.rcdefaults()
epochs=range(0, 40, 1)
plt.plot(epochs, train_acc, color='#006BA4')
plt.plot(epochs, val_acc, color='#FF800E')
plt.grid(b=True, which='major', color='lightgray')
plt.grid(b=True, which='minor', color='lightgray')
plt.xticks(np.arange(0, 45, 5))
plt.yticks(np.arange(0.5, 1, 0.05))
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 600
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['Training Acc.', 'Validation Acc.'],loc='lower right')

In [ ]:
matplotlib.rcdefaults()
#This code block draw graph for loss
plt.plot(epochs, train_losses, color='#006BA4')
plt.plot(epochs, val_losses, color='#FF800E')
plt.grid(b=True, which='major', color='lightgray')
plt.grid(b=True, which='minor', color='lightgray')
plt.xticks(np.arange(0, 45, 5))
plt.yticks(np.arange(0, 1.2, 0.2))
plt.rcParams['figure.dpi'] = 600
#plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['Training Loss', 'Validation Loss'],loc='lower right')

# Model Test

In [ ]:
get_metric.test_model(ensemble_1,device,dataloaders['val'])

In [ ]:
actuals, predictions = get_metric.test_label_predictions(ensemble_1, device, dataloaders['val'])

In [ ]:
get_metric.get_classification_report(actuals, predictions)

In [ ]:
get_metric.get_confusion_matrix(actuals, predictions)

In [ ]:
get_metric.get_cohen_kappa(actuals, predictions)

In [ ]:
get_metric.get_roc_curves(ensemble_1, device,  dataloaders['val'])